In [3]:
def get_genome_file(genome_location,species):
    list_of_files_in_genome_folder = os.listdir(f"{genome_location}/{species}")
    for file in list_of_files_in_genome_folder:
        if file.endswith("_genomic.fna"):
            genome_file = file
    return(genome_file)  

def get_gene_sequence(genome_location, 
                      species, 
                      genome_file, 
                      scaffold,
                      gene_start,
                      gene_end,
                      complement,
                      output_location):

    
    print("Getting Gene")
    list_of_folders = os.listdir(output_location)
    # print(list_of_files_inside_annotated_species_folder)
    # print(
    if f"0.Temp" not in list_of_folders:
        os.mkdir(f"{output_location}/0.Temp")

#     subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}"', shell = True, stderr = subprocess.DEVNULL)
#     print(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end}')
    subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end} > "{output_location}/0.Temp/temp_genome.fa"', shell = True, stderr = subprocess.DEVNULL)
    
    genome = SeqIO.parse(f"{output_location}/0.Temp/temp_genome.fa", "fasta")
    for entries in genome:
        gene_sequence = entries.seq
        if complement == "1":
            gene_sequence = gene_sequence.reverse_complement()
        break
    # print(gene_sequence)
    return (gene_sequence)
def make_output_folder(output_location):
    list_of_folders = os.listdir(output_location)
    if "2.Final_output" in list_of_folders:
        subprocess.run(f'rm -r "{output_location}/2.Final_output"', shell = True, stderr = subprocess.DEVNULL)
    os.mkdir(f"{output_location}/2.Final_output")

def write_exon_output(output_location,species,exon,exon_output):
    
    
    list_of_folders_in_final_output = os.listdir(f"{output_location}/2.Final_output")
    if species not in list_of_folders_in_final_output:
        os.mkdir(f"{output_location}/2.Final_output/{species}")
    
    with open(f"{output_location}/2.Final_output/{species}/{exon}.fa", 'w') as out_file:
        out_file.write(exon_output)

In [5]:
import os
from Bio import SeqIO
import subprocess

family_group = "8.Lycaenidae"


output_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/9.CLOCK Exon Analysis/{family_group}"
make_output_folder(output_location)
genome_location = "/mnt/f/Genomes_2023-12-26"
list_of_species = os.listdir(f"{output_location}/1.Blast_result")
if 'desktop.ini' in list_of_species:
    list_of_species.remove("desktop.ini")
# print(list_of_species)
# list_of_species = ["Heliconius_charithonia"]
# list_of_species = ["Maniola_hyperantus"]
error_exons = ''
for species in list_of_species:
    print(species)
    genome_file = get_genome_file(genome_location,species)
    print(genome_file)
    previous_coordinate_start = ''
    previous_scaffold = ''
    with open(f"{output_location}/1.Blast_result/{species}/final_coordinates.csv", 'r') as coordinate_final_file:
        coordinate_file_list = coordinate_final_file.readlines()
    last_exon = len(coordinate_file_list)-1
    
    for i in range(1,len(coordinate_file_list)):
        
        coordinate_line_split = coordinate_file_list[i].split(",")
        
            
#         print(coodinate_line_split[5])
        if coordinate_line_split[5] == "N":
            if previous_coordinate_start == '':
                previous_coordinate_start = coordinate_line_split[2]
                previous_scaffold = coordinate_line_split[1]
            complement = coordinate_line_split[4]
            start,stop = coordinate_line_split[2],coordinate_line_split[3]
            print(start,stop)
            query_name = coordinate_line_split[6]
            scaffold = coordinate_line_split[1]
            exon = f"Exon_{query_name.split('_')[-1]}"
            left_oh, right_oh = query_name.split('_')[-6], query_name.split('_')[-4]
            print(left_oh, right_oh)
#             print(complement)
            if complement == "0":
                if start < previous_coordinate_start or scaffold != previous_scaffold:
                    print(f"Error in Coordinates\nPrevious_coordinate = {coordinate_file_list[i-1]}\nCurrent_coordinate = {coordinate_file_list[i]}")
                    prompt = input("Continue (Y/N)?")
                    if prompt[0].lower() == "n":
                        assert False
            elif complement == "1":
                if start > previous_coordinate_start or scaffold != previous_scaffold:
                    print(f"Error in Coordinates\nPrevious_coordinate = {coordinate_file_list[i-1]}\nCurrent_coordinate = {coordinate_file_list[i]}")
                    prompt = input("Continue (Y/N)?")
                    if prompt[0].lower() == "n":
                        assert False
            else:
                print("ERROR")
            
        elif coordinate_line_split[5] == "Y":
            query_name = coordinate_line_split[6]
            exon = f"Exon_{query_name.split('_')[-1]}"
            scaffold = coordinate_line_split[1]
            error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
            continue
            
        gene_sequence = get_gene_sequence(genome_location, 
                                          species, 
                                          genome_file, 
                                          scaffold,
                                          start,
                                          stop,
                                          complement,
                                          output_location)
        
       
        current_exon = query_name.split('_')[-1]
        print(f"Current Exon = {current_exon}")
        if current_exon == "1":
            if complement == 1:
                translated = gene_sequence.reverse_complement.translate()
            else:
                translated = gene_sequence.translate()
            print(translated)
            if "*" in translated and translated[0] != "M":
                error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
            else:
                exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
                print(exon_output)
        
                write_exon_output(output_location,species,exon,exon_output)
        elif current_exon == f"{last_exon}":
            if complement == 1:
                translated = gene_sequence[:-int(left_oh)].reverse_complement.translate()
                print(gene_sequence)
            else:
                translated = gene_sequence[int(left_oh):].translate()
                print(gene_sequence)

            if "*" in translated[:-1] and translated[-1] != "*":
                error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
                print(translated)
            else:
                
                exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
                print(exon_output)
                write_exon_output(output_location,species,exon,exon_output)
#         assert False
        else:
            exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
            print(exon_output)

            write_exon_output(output_location,species,exon,exon_output)
        
#         assert False
        previous_coordinate_start = start
        previous_scaffold = scaffold
    error_exons += "\n\n"
#     print(coordinate_file_list)
#     break
coordinate_file = ''
print(error_exons)
with open(f"{output_location}/error_exons.txt", 'w') as error_out_file:
    error_out_file.write(error_exons)

Aricia_agestis
GCF_905147365.1_ilAriAges1.1_genomic.fna
32774692 32774729
0 2
Getting Gene
Current Exon = 1
MDDDGDDKDDTK
>Aricia_agestis_Exon_1_NC_056428.1_32774692_32774729_left_0_right_2
ATGGATGACGACGGAGACGACAAAGATGACACAAAACG
32771356 32771504
1 1
Getting Gene
Current Exon = 2
>Aricia_agestis_Exon_2_NC_056428.1_32771356_32771504_left_1_right_1
GCGAACACGTAACCTTAGCGAGAAGAAGAGGAGGGATCAGTTTAACATGCTTGTTAATGAGCTTGGATCTATGATCTCCTCAAGTAACCGCCGAATGGACAAATCTACTGTGCTCAAGTCCACAATATCATTCCTGAAAAATCATAATG
32771022 32771203
2 0
Getting Gene
Current Exon = 3
>Aricia_agestis_Exon_3_NC_056428.1_32771022_32771203_left_2_right_0
AAGTTACTATCCGTTCGCGAGTACACGACGTCCAAGAAGATTGGAAGCCAActtttctgtcaaatgaagAGTTTACATATTTAATGCTCGAAGCCCTAGAGGGTTTTATCATGGTGCTGGCACCAGACGGTACAATTCATTACGTCTCCGAGAGCATCACTTCGCTACTGGGACATAGTTCT
32769986 32770103
0 1
Getting Gene
Current Exon = 4
>Aricia_agestis_Exon_4_NC_056428.1_32769986_32770103_left_0_right_1
AACGACATTATCCAGACAAATTTATTTGATTTAACATATCAAGACGATCGCTCGAATCTGTACAATATCTTACAAAATA

14021694 14021789
2 1
Getting Gene
Current Exon = 5
>Celastrina_argiolus_Exon_5_LR994583.1_14021694_14021789_left_2_right_1
AGACCGAGATTCGCTTCCAGTGTCACTTGCAGCGAGGAACCCTGGACTTCCGAGACTGTGTCAACTATGAGCTCATAGAGTTCAATGGACACTTCC
14007797 14007917
2 2
Getting Gene
Current Exon = 6
>Celastrina_argiolus_Exon_6_LR994583.1_14007797_14007917_left_2_right_2
GTATGTTGCAGTATATTGATCGGACTGACAATGAGGAGCACTCAGGGTATCAGCGGGAACAGGAAACCAGGCAAGTACTTTTATGTTACCTTTGTATTTCAATCCATAAGGGTTACTACAG
14005624 14005862
1 1
Getting Gene
Current Exon = 7
>Celastrina_argiolus_Exon_7_LR994583.1_14005624_14005862_left_1_right_1
GTTACTGTTCGTCTGTACGGGAAGACTTCACACGCCTCATTTGATACGAGACGTATCACTAGTGGACATAAATCGAAACGAATTCACGTCCAGACATAGTCTGGAATGGAAGTTCCTGTTCCTGGATCACCGCGCGCCGCCCATAATAGGATACTTGCCCTTCGAAGTGCTTGGCACATCAGGATACGATTATTACCACTTTGACGATCTGGAAAAAGTCGTGACGTGCCATGAAGCTC
14005048 14005195
2 2
Getting Gene
Current Exon = 8
>Celastrina_argiolus_Exon_8_LR994583.1_14005048_14005195_left_2_right_2
TAATGCAGAAAGGTGAGCTTACGTCTTGCTACTATCGGTTCCTGA

Current Exon = 9
>Eumaeus_atala_Exon_9_JAFELO010000500.1_647002_647149_left_1_right_0
CTATGCTGACATCATGAAGAATACGAAACGCGAGCAGGCGGAGTCGGTGCGACCTGTAGGGGCAGAGCCGAACGAGGAGGTTCTGGAGACTCCGAACTCTGATAACATCATGGACACGTTCTGCGACTCCTACCGTACTATGTCTCAG
649199 649354
0 0
Getting Gene
Current Exon = 10
>Eumaeus_atala_Exon_10_JAFELO010000500.1_649199_649354_left_0_right_0
GCGTCCTCGGTGAAGTCCACCTCTTGCAGCACCACCGGGACGCCGACCGCCTCAGGCAGCAACCCCTGGTCTAGAGTCAACTCGCAGGCCTGCTACAACTCGGAGTCATCAGCCGCCTCCAACGAGTCGAGGTCGTCGCAGCACACGCAGCTTGAT
649834 649989
0 0
Getting Gene
Current Exon = 11
>Eumaeus_atala_Exon_11_JAFELO010000500.1_649834_649989_left_0_right_0
GGCAACAAAGTATTGGAGCCGATTTTGGTGCCACTGCACGGTATTGGTGCTCAGTATCTCCATCCAGCTCCCTACGTGGCGGCTCTGAGTGTGCCAAGCGTGTTGGCACAACCCTCTTTGCCTGTCGTCATGACTCATGACCAGACACAAATGCAG
652584 652706
0 0
Getting Gene
Current Exon = 12
>Eumaeus_atala_Exon_12_JAFELO010000500.1_652584_652706_left_0_right_0
GGGCAGTTGCAACGCAAGCACGAGGAACTGCAGCAGATGATTGTGCGCCAGCAGGAGGAGTTGCGCCAGGTCAAGGAGCAGCTGTTACTGGCGA

10373752 10373913
0 0
Getting Gene
Current Exon = 13
GTATTTCTTCATTTACAGTACCACGAAGAAGCACCATCAAATCAACGTGTGCCGGCACAAATACTTTACGAAGAGATACAGAATAGAAACCCCCCGGGCTACCAACCATACCCACCGTCGGACGGGACGAACCAACAACAAAATCCAGATATGCCGCGGTAA
>Helleia_helle_Exon_13_OY971420.1_10373752_10373913_left_0_right_0
GTATTTCTTCATTTACAGTACCACGAAGAAGCACCATCAAATCAACGTGTGCCGGCACAAATACTTTACGAAGAGATACAGAATAGAAACCCCCCGGGCTACCAACCATACCCACCGTCGGACGGGACGAACCAACAACAAAATCCAGATATGCCGCGGTAA
Lycaena_phlaeas
GCA_905333005.2_ilLycPhla1.2_genomic.fna
8145845 8145882
0 2
Getting Gene
Current Exon = 1
MDDDGDDKDDTK
>Lycaena_phlaeas_Exon_1_HG995168.1_8145845_8145882_left_0_right_2
ATGGATGACGACGGAGATGACAAAGATGATACGAAACG
8141948 8142096
1 1
Getting Gene
Current Exon = 2
>Lycaena_phlaeas_Exon_2_HG995168.1_8141948_8142096_left_1_right_1
GCGGACACGTAACCTGAGCGAGAAGAAGAGGAGAGACCAATTTAATATGCTCGTAAGTGAGCTAGGATCTATGATTTCATCCAGTAATCGGAGAATGGATAAATCGACTGTACTCAAATCTACGATATCATTTCTAAGAAATCACAATG
8141122 8141303
2 0
Getting Gene
Current Exon = 3
>Lycaena_phla

22361506 22361623
0 1
Getting Gene
Current Exon = 4
>Lysandra_coridon_Exon_4_HG992055.1_22361506_22361623_left_0_right_1
aACGACATTATCCAGACAAATCTATTTGATTTAACATATCAAGACGATCGCTCGAATCTGTACAATATCTTACAAAATAATACCACAGCGGCCGATTCAACTGTAACACAGTCAGCAG
22358565 22358660
2 1
Getting Gene
Current Exon = 5
>Lysandra_coridon_Exon_5_HG992055.1_22358565_22358660_left_2_right_1
AAAATGAAATTCGGTTCCAATGTCACTTGCGTAGAGGAACGTTGGATTTCCGAGACAACGTCAACTATGAACTCGTAGAGTTCAACGGACACTTCC
22353652 22353697
2 2
Getting Gene
Current Exon = 6
>Lysandra_coridon_Exon_6_HG992055.1_22353652_22353697_left_2_right_2
GTATGTTACAGTATATTGAACAGACAGATAATGAGGAGCATTCAGG
22350109 22350347
1 1
Getting Gene
Current Exon = 7
>Lysandra_coridon_Exon_7_HG992055.1_22350109_22350347_left_1_right_1
GTTACTCTTCGTATGTACGGGCAGGCTTCACACGCCGCATCTAATACGCGACGTCTCACTGGTTGACCTCAATCGAAGCGAGTTCACATCTAGACATAGTCTAGAATGGAAGTTCCTGTTCCTGGATCATCGTGCGCCGCCTATAATAGGATACCTGCCGTTCGAAGTGTTGGGCACCTCCGGATACGATTATTACCACTTCGACGATCTGGAGAAGGTGGTTACGTGTCACGAAGCAT
22349284 2234

Current Exon = 8
>Plebejus_argus_Exon_8_FR989932.1_7602410_7602557_left_2_right_2
TGATGCAGAAAGGCGAGCTAACGTCTTGCTACTATAGGTTCCTGACGAAGGGTCAGCAGTGGATCTGGCTGCAGACCCGGTTCTACATCACATACCACCAGTGGAATGCAAAACCCGAGTTCGTTGTTTGTACGCATCGAGTCGTCAG
7616111 7616294
1 0
Getting Gene
Current Exon = 9
>Plebejus_argus_Exon_9_FR989932.1_7616111_7616294_left_1_right_0
TTACGCCGACATCGTGAAGAGTACGAAGCAAGAGAGCGCGGAGTCGGAACATGTGACAGAAGTGGACCAAAACGAGGACATGTGTGACAATCCCAGCTCGGACAACGTCCTGGACACGATATCGTCATCGTACATGACTGAGAACAGCGAGACATACGGTGGCTCGTATCATACTCTATCGCAG
7616932 7617102
0 0
Getting Gene
Current Exon = 10
>Plebejus_argus_Exon_10_FR989932.1_7616932_7617102_left_0_right_0
CCGTCGTCGGTAAAATCAACGTCGGGCAGCACGACGGGCACGCCGACGGCGTCGGGCAGCGCCGCGTGGTCGCGCGTCACGTCGCAGGTGTGCTTCAACTCCGAGTCCGACCGCGCCTCCATGTCCAGCGAGTCGCGATCCTCGCATCGGAACAGCACGAGAGAGAGAGAG
7620842 7621003
0 0
Getting Gene
Current Exon = 11
>Plebejus_argus_Exon_11_FR989932.1_7620842_7621003_left_0_right_0
CATCAAAGCAAGTTGGAGCCGATCCTGGTGCCGCTGCACGGCATCGGCACTCAGTACTTGCAG

Current Exon = 11
>Polyommatus_iphigenia_Exon_11_OY730178.1_30216666_30216827_left_0_right_0
TATCAGAGCAAGTTGGAGCCGATTCTGGTGCCGCTGCACGGCATCGGCACTCAGTACTTGCAGCCGGCGCCGTACGTCGGCACGGTGTGCGTGCCCACCGCCCCGCCCctcgcgccgctcccgcccctcTCCGTCATGACGCACGATCAGACGCAGATGCAG
30214712 30214834
0 0
Getting Gene
Current Exon = 12
>Polyommatus_iphigenia_Exon_12_OY730178.1_30214712_30214834_left_0_right_0
GAGCAGTTGCAGCGGAAGCATGAAGAGCTCCAGCAGATGATAGTGCGCCAGCAGGAGGAGCTGCGCCAGGTGAAGGAGCAGCTGCTGCTGGCCAGGCTGGGCATCCTGCAGCCCGTCATCAAC
30213413 30213571
0 0
Getting Gene
Current Exon = 13
GTTCCAAATTCGTACGGCAACCTAGAGGAAGTTCCGCCGAACCAGCGCATACCCCCACAGATACTGTACGAGGAGGCACCACCGCGCAATATCACGTACGCGCAACAAATGCCCCCCAACAACGGAAACAATCAACAGAACCCAAACATGCCGCGATAA
>Polyommatus_iphigenia_Exon_13_OY730178.1_30213413_30213571_left_0_right_0
GTTCCAAATTCGTACGGCAACCTAGAGGAAGTTCCGCCGAACCAGCGCATACCCCCACAGATACTGTACGAGGAGGCACCACCGCGCAATATCACGTACGCGCAACAAATGCCCCCCAACAACGGAAACAATCAACAGAACCCAAACATGCCGCGATAA





























